In [ ]:
# @title z-image-base - INT8 Per-Channel 量化（Turbo 完全对齐版）

# ==========================================
# 步骤 1：安装依赖
# ==========================================
!pip install -U -q git+https://github.com/Disty0/sdnq git+https://github.com/huggingface/diffusers
!pip install -U -q transformers accelerate safetensors psutil

# ==========================================
# 步骤 2：主量化流程
# ==========================================
import os
import json
import gc
import shutil
import torch
from google.colab import drive
from safetensors.torch import save_file, load_file
from huggingface_hub import hf_hub_download
import psutil

def print_mem():
    mem = psutil.virtual_memory()
    used_gb = mem.used / 1e9
    total_gb = mem.total / 1e9
    percent = mem.percent
    print(f"💻 RAM: {used_gb:.1f}GB / {total_gb:.1f}GB ({percent:.1f}%)")
    if percent > 85:
        print("⚠️  内存使用超过 85%，强制 GC")
        gc.collect()

def force_cleanup():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# 挂载 Drive
if not os.path.exists("/content/drive"):
    drive.mount("/content/drive")

FINAL_DIR = "/content/drive/MyDrive/z-image-base-transformer-int8-per-channel"
LOCAL_DIR = "/content/quant_output"
TEMP_DIR = "/content/temp_quant"

for d in [LOCAL_DIR, TEMP_DIR]:
    shutil.rmtree(d, ignore_errors=True)
    os.makedirs(d, exist_ok=True)

# 与 Turbo 完全一致的排除列表
target_modules_to_not_convert = [
    "layers.0.adaLN_modulation.0.weight",
    "all_x_embedder",
    "t_embedder",
    "cap_embedder",
    "all_final_layer"
]

def should_quantize(name):
    """判断是否量化（与 Turbo 逻辑一致）"""
    if name in target_modules_to_not_convert:
        return False

    for skip_prefix in target_modules_to_not_convert:
        if name.startswith(skip_prefix):
            return False

    return name.endswith(".weight")

def quantize_tensor_int8_per_channel(tensor):
    """
    INT8 Per-Channel 对称量化（与 Turbo 完全对齐）

    对于 2D 权重 [out_features, in_features]：
    - 对每个输出通道（dim=0）独立计算 scale
    - 返回 scale shape: [out_features, 1]（与 Turbo 一致）

    对于 1D 权重（如 bias/norm）：
    - 使用 per-tensor 量化
    """
    if tensor.dtype == torch.int8:
        return tensor, None

    t_float = tensor.float()

    # 🔧 关键：2D 权重用 per-channel，1D 用 per-tensor
    if tensor.dim() == 2:
        # Per-Channel 量化
        # 沿 dim=1 求最大值，保留 dim=0（输出通道）
        abs_max_per_channel = t_float.abs().amax(dim=1, keepdim=True)  # shape: [out_features, 1]

        # 避免除零
        abs_max_per_channel = torch.clamp(abs_max_per_channel, min=1e-8)

        # 计算 scale（每个输出通道一个）
        scale = abs_max_per_channel / 127.0  # shape: [out_features, 1]

        # 量化（自动广播）
        q_tensor = (t_float / scale).round().clamp(-128, 127).to(torch.int8)

        # 🔧 关键：保持 scale 的 shape 为 [out_features, 1]（与 Turbo 一致）
        return q_tensor, scale.to(torch.bfloat16)

    else:
        # Per-Tensor 量化（用于 1D/3D+ 权重）
        abs_max = t_float.abs().max().item()

        if abs_max == 0 or abs_max < 1e-8:
            return tensor.to(torch.int8), torch.tensor(1.0, dtype=torch.bfloat16)

        scale = abs_max / 127.0
        q_tensor = (t_float / scale).round().clamp(-128, 127).to(torch.int8)

        # 🔧 返回标量 scale
        return q_tensor, torch.tensor(scale, dtype=torch.bfloat16)

print("="*60)
print("🚀 开始 Per-Channel 量化流程（Turbo 完全对齐）")
print("="*60)
print_mem()

# ==========================================
# 1. 下载原始模型
# ==========================================
print("\n📥 步骤 1: 下载模型索引...")

index_file = hf_hub_download(
    "Tongyi-MAI/Z-Image",
    "transformer/diffusion_pytorch_model.safetensors.index.json",
    cache_dir=TEMP_DIR,
    resume_download=True
)

with open(index_file, "r") as f:
    original_index = json.load(f)

weight_map = original_index["weight_map"]
shard_files = sorted(set(weight_map.values()))

print(f"   ✓ 原始分片数: {len(shard_files)}")
print(f"   ✓ 总参数数: {len(weight_map)}")

config_file = hf_hub_download(
    "Tongyi-MAI/Z-Image",
    "transformer/config.json",
    cache_dir=TEMP_DIR,
    resume_download=True
)

with open(config_file, "r") as f:
    base_config = json.load(f)

print(f"   ✓ config.json 已下载")
print_mem()

# ==========================================
# 2. 逐分片量化
# ==========================================
print(f"\n🔄 步骤 2: 逐分片量化...")

TEMP_SHARD_SIZE = 500_000_000  # 500MB

temp_shard_idx = 1
current_temp_shard = {}
current_temp_size = 0
scales_dict = {}

quantized_count = 0
kept_count = 0

for shard_idx, shard_name in enumerate(shard_files):
    print(f"\n{'─'*60}")
    print(f"📦 分片 {shard_idx+1}/{len(shard_files)}: {shard_name}")
    print_mem()

    shard_path = hf_hub_download(
        "Tongyi-MAI/Z-Image",
        f"transformer/{shard_name}",
        cache_dir=TEMP_DIR,
        resume_download=True
    )

    shard_data = load_file(shard_path)
    print(f"   包含参数: {len(shard_data)} 个")

    for param_idx, (param_name, tensor) in enumerate(shard_data.items()):
        tensor = tensor.cpu()

        if should_quantize(param_name) and tensor.dim() >= 2:
            # 🔧 使用 Per-Channel 量化
            q_tensor, scale = quantize_tensor_int8_per_channel(tensor)

            if scale is not None:
                # 🔧 关键：scale 键名格式
                scale_key = param_name.replace(".weight", ".scale")
                scales_dict[scale_key] = scale

                quantized_count += 1

                if param_idx < 3:
                    print(f"   ✓ [{param_idx+1}] {param_name}")
                    print(f"      量化: {list(tensor.shape)} → int8")
                    print(f"      scale shape: {list(scale.shape)} (per-channel)")

        else:
            # 保持为 bfloat16
            q_tensor = tensor.to(torch.bfloat16) if tensor.dtype != torch.bfloat16 else tensor
            kept_count += 1

            if param_idx < 3:
                print(f"   - [{param_idx+1}] {param_name}")
                print(f"      保持: bfloat16")

        param_size = q_tensor.numel() * q_tensor.element_size()

        # 内存控制：达到临时分片大小就保存
        if current_temp_size + param_size > TEMP_SHARD_SIZE and current_temp_shard:
            temp_file = os.path.join(TEMP_DIR, f"temp_{temp_shard_idx:04d}.safetensors")
            print(f"\n   💾 暂存临时分片 #{temp_shard_idx} ({len(current_temp_shard)} 参数)")
            save_file(current_temp_shard, temp_file)

            current_temp_shard.clear()
            current_temp_size = 0
            temp_shard_idx += 1
            force_cleanup()

        current_temp_shard[param_name] = q_tensor
        current_temp_size += param_size

        del tensor, q_tensor

    del shard_data
    force_cleanup()

    try:
        os.remove(shard_path)
    except:
        pass

# 保存最后一个临时分片
if current_temp_shard:
    temp_file = os.path.join(TEMP_DIR, f"temp_{temp_shard_idx:04d}.safetensors")
    print(f"\n💾 保存最后临时分片 #{temp_shard_idx}")
    save_file(current_temp_shard, temp_file)
    del current_temp_shard
    force_cleanup()

total_temp_shards = temp_shard_idx

# 保存 scales
if scales_dict:
    print(f"\n💾 保存量化 scales: {len(scales_dict)} 个")

    # 🔧 验证：检查 scale 的 shape
    sample_scale_keys = list(scales_dict.keys())[:3]
    print(f"   前 3 个 scale 的 shape:")
    for k in sample_scale_keys:
        print(f"      {k}: {list(scales_dict[k].shape)}")

    save_file(scales_dict, os.path.join(TEMP_DIR, "temp_scales.safetensors"))
    del scales_dict
    force_cleanup()

print(f"\n{'='*60}")
print(f"✅ 量化统计:")
print(f"   量化参数: {quantized_count}")
print(f"   保持参数: {kept_count}")
print(f"   临时分片数: {total_temp_shards}")
print(f"{'='*60}")
print_mem()

# ==========================================
# 3. 合并为单个文件
# ==========================================
print(f"\n🔗 步骤 3: 合并为单个文件...")

final_state_dict = {}

for i in range(1, total_temp_shards + 1):
    temp_file = os.path.join(TEMP_DIR, f"temp_{i:04d}.safetensors")

    if not os.path.exists(temp_file):
        continue

    print(f"   [{i}/{total_temp_shards}] 加载临时分片...")
    temp_data = load_file(temp_file)
    final_state_dict.update(temp_data)
    del temp_data
    force_cleanup()

    if i % 5 == 0:
        print_mem()

# 加载 scales
scales_file = os.path.join(TEMP_DIR, "temp_scales.safetensors")
if os.path.exists(scales_file):
    print(f"   加载 scales...")
    scales_data = load_file(scales_file)
    final_state_dict.update(scales_data)
    del scales_data
    force_cleanup()

print(f"\n✅ 合并统计:")
print(f"   总参数数: {len(final_state_dict)}")

# 验证
scale_keys = [k for k in final_state_dict.keys() if k.endswith('.scale')]
weight_keys = [k for k in final_state_dict.keys() if k.endswith('.weight')]

print(f"   权重参数: {len(weight_keys)}")
print(f"   Scale 参数: {len(scale_keys)}")

# 🔧 验证 scale 的 shape
print(f"\n   验证 scale shape（前 3 个）:")
for k in sorted(scale_keys)[:3]:
    scale = final_state_dict[k]
    print(f"      {k}: {list(scale.shape)}")

# 保存单文件
print(f"\n💾 保存最终文件...")

save_file(
    final_state_dict,
    os.path.join(LOCAL_DIR, "diffusion_pytorch_model.safetensors"),
    metadata={"format": "pt"}
)

file_size = os.path.getsize(os.path.join(LOCAL_DIR, "diffusion_pytorch_model.safetensors"))
print(f"   ✓ 文件大小: {file_size/1e9:.2f} GB")

del final_state_dict
force_cleanup()
print_mem()

# ==========================================
# 4. 生成配置文件
# ==========================================
print(f"\n📝 步骤 4: 生成配置文件...")

quantization_config_standalone = {
    "add_skip_keys": False,
    "dequantize_fp32": False,
    "group_size": -1,
    "is_integer": True,
    "is_training": False,
    "modules_dtype_dict": {},
    "modules_to_not_convert": target_modules_to_not_convert,
    "non_blocking": False,
    "quant_conv": False,
    "quant_method": "sdnq",
    "quantization_device": None,
    "quantized_matmul_dtype": None,
    "return_device": None,
    "sdnq_version": "0.1.2",
    "svd_rank": 32,
    "svd_steps": 8,
    "use_grad_ckpt": True,
    "use_quantized_matmul": False,
    "use_quantized_matmul_conv": False,
    "use_static_quantization": True,
    "use_stochastic_rounding": False,
    "use_svd": False,
    "weights_dtype": "int8"
}

with open(os.path.join(LOCAL_DIR, "quantization_config.json"), "w") as f:
    json.dump(quantization_config_standalone, f, indent=2)

quantization_config_embedded = {
    **quantization_config_standalone,
    "quantization_device": "xpu",
    "return_device": "cpu",
    "add_skip_keys": True,
}

base_config["quantization_config"] = quantization_config_embedded
base_config["_diffusers_version"] = "0.36.0.dev0"
base_config.pop("siglip_feat_dim", None)

with open(os.path.join(LOCAL_DIR, "config.json"), "w") as f:
    json.dump(base_config, f, indent=2)

print("   ✓ 配置文件已生成")

# ==========================================
# 5. 拷贝到 Drive
# ==========================================
print(f"\n📤 步骤 5: 拷贝到 Drive...")
print(f"   目标: {FINAL_DIR}")

if os.path.exists(FINAL_DIR):
    shutil.rmtree(FINAL_DIR)

shutil.copytree(LOCAL_DIR, FINAL_DIR)
print("   ✓ 拷贝完成")

print(f"\n🧹 清理临时文件...")
shutil.rmtree(LOCAL_DIR, ignore_errors=True)
shutil.rmtree(TEMP_DIR, ignore_errors=True)
force_cleanup()

# ==========================================
# 6. 最终验证
# ==========================================
print(f"\n{'='*60}")
print(f"🎉 Per-Channel 量化完成！")
print(f"{'='*60}")

print(f"\n📁 最终文件:")
for f in sorted(os.listdir(FINAL_DIR)):
    fpath = os.path.join(FINAL_DIR, f)
    if os.path.isfile(fpath):
        fsize = os.path.getsize(fpath)
        if fsize > 1e6:
            print(f"   ✓ {f:<45} {fsize/1e9:>8.2f} GB")
        else:
            print(f"   ✓ {f:<45} {fsize/1e3:>8.1f} KB")

print(f"\n✅ 与 Turbo 对齐检查:")
print(f"   ✓ 量化方式: Per-Channel")
print(f"   ✓ Scale shape: [out_features, 1]")
print(f"   ✓ Scale dtype: bfloat16")
print(f"   ✓ 未量化层: bfloat16")
print(f"   ✓ 量化参数数: {quantized_count}")

print(f"\n🚀 使用方法:")
print(f"   !cp -rf {FINAL_DIR}/* \\")
print(f"           /content/Z-Image-Turbo-SDNQ-int8/transformer/")

print_mem()